In [ ]:
# Only need to do this once...
!pip install flask
!pip install --upgrade google-cloud-datastore
!pip install Flask-Mail

In [ ]:

import json
import re
from flask import Flask, request, jsonify, make_response
from flask import render_template
from google.cloud import datastore

In [ ]:
from send_invitation import send_email 
from threading import Thread
import jwt

app = Flask(__name__)
results = ""
rsize = 0

import re
from random import seed
from random import randint
from datetime import datetime

seed(datetime.now())

app.config['SECRET_KEY'] = 'SECRET_KEY_CHAT9900'

def get_public_url():
    '''
    tunnel = !curl https://4040-44eb2645-47f4-496a-9df1-70647bdfe449.cs-asia-southeast1-vjqr.cloudshell.dev/api/tunnels
    tunnel= str(tunnel)
    mobj = re.search('public_url', tunnel)
    i = mobj.span()
    t = tunnel[i[1]:].split('\"')
    public_url =t[2]
    '''
    # populate this from your local server
    public_url = "http://75789421033d.ngrok.io"
    return public_url

def gen_jwt():
    return jwt.encode(
            {'chatroom': 101 },
            app.config['SECRET_KEY'], algorithm='HS256')

@app.route('/chatroom/<token>', methods=['GET', 'POST'])
def reset_password(token):
    meeting = jwt.decode(token, app.config['SECRET_KEY'],
                            algorithms=['HS256'])['chatroom']
    print("meeting click was clicked ",meeting)
    return render_template('place_holder.html')

def send_password_reset_email():
    #user should have username and email
    user = "Test User"
    token = jwt.encode(
            {'reset_password': 101 },
            app.config['SECRET_KEY'], algorithm='HS256')

@app.route('/webhook/', methods=['POST'])
def handle():
    req = request.get_json(silent=True, force=True)
    print('Request:')
    print(json.dumps(req, indent=4))
    topic = req.get('queryResult').get('parameters').get('topic')
    intent = req.get('queryResult').get('intent').get('displayName')
    session = req.get('queryResult').get('outputContexts')[0].get('name')
    print(topic)
    # there could be multiple topics in a sentence
    if isinstance(topic, list):
        topic = topic[0]
    #topic = re.sub(r'[^\w\s]', '', topic)
    topic = topic.lower()
    print("topic is ",topic)
    
    rsp = buildReply('Command not recognized')
    if (topic == "schedule"):
        rsp = scheduleMovie(req)
    elif (topic == "profile" or topic == "account"):
        rsp = getProfileAccount()
    elif (topic == "movie recommendation" ):
        rsp = getRecommendations()
    elif (topic == "login"):
        username = req.get('queryResult').get('parameters').get("username")
        password = req.get('queryResult').get('parameters').get("password")
        rsp = validateLogin(username, password, session)
        
    rsp = json.dumps(rsp, indent=4)
    print (rsp)
    r = make_response(rsp)
    r.headers['Content-Type'] = 'application/json'
    return r    
  
def scheduleMovie(req):
    confirm = req.get('queryResult').get('parameters').get('confirm').lower()
    print(confirm)
    if ("yes" in confirm):
        attendees = ["chatbot9900@gmail.com", "e.life.hacker@gmail.com"]
        public_url = get_public_url()
        token = gen_jwt()
        chatroom_url = public_url+"/"+token.decode("utf-8")
        Thread(target=send_email, args=(attendees,chatroom_url,)).start()
        return buildReply('Sending request ...')
    return buildReply('Aborted')
  
def getProfileAccount():
    urls = ""
    with open('netflix_urls.txt') as netflix_urls:
        urls = netflix_urls.read()
    print (urls)
    return buildReply(urls)

def getRecommendations():
    global results 
    global rsize
    print ("recom")
    if (rsize ==0):
        client = datastore.Client()
        query = client.query(kind='netflix')
        results = list(query.fetch(limit=100))
        rsize = len(results)
        print(rsize)
    t1 = results[randint(0, rsize -1 )]['title'].decode("utf-8") 
    print("t1",t1)
    t2 = results[randint(0, rsize-1)]['title'].decode("utf-8") 
    return buildReply(t1+"\n"+t2)

def validateLogin(un, pw, sn):
    client = datastore.Client()
    # you may have to create this kind manually in gcp datastore 
    query = client.query(kind="users")
    print(un,pw)
    # These two columns has to be indexed in gcp datastore 
    # otherwise they can not be found and filtered
    query.add_filter("username", "=", un)
    query.add_filter("password", "=", pw)
    result = list(query.fetch(1))
    print("validateLogin",result)
    if len(result) == 1:
        return confirmLogin(un, sn)
    return buildReply("Invalid combination")
  
def confirmLogin(un, sn):
    return {
      "fulfillmentText": 
              "User: "+un+" logged in"
      ,
      "outputContexts": [
        {
          "name": sn,
          "lifespanCount": 5,
          "parameters": {
            "loggedin_user": un
          }
        }
      ]
    }

def buildReply(info):
    return {
        'fulfillmentText': info,
    }

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Mar/2021 05:29:23] "POST /webhook/ HTTP/1.1" 200 -


Request:
{
    "queryResult": {
        "intent": {
            "displayName": "login-prompt",
            "name": "projects/i-backbone-307006/locations/global/agent/intents/9416bbd6-b6b2-472e-b1c1-119a3e368726"
        },
        "parameters": {
            "topic": "login",
            "password": "test123",
            "username": "elife"
        },
        "sentimentAnalysisResult": {
            "queryTextSentiment": {
                "magnitude": 0.3,
                "score": 0.3
            }
        },
        "intentDetectionConfidence": 0.3,
        "queryText": "test123",
        "languageCode": "en",
        "allRequiredParamsPresent": true,
        "outputContexts": [
            {
                "parameters": {
                    "no-input": 0.0,
                    "topic": "login",
                    "username": "elife",
                    "password.original": "test123",
                    "username.original": "elife",
                    "no-match": 0.0,
         

127.0.0.1 - - [27/Mar/2021 05:29:44] "POST /webhook/ HTTP/1.1" 200 -


Request:
{
    "queryResult": {
        "intent": {
            "displayName": "login-prompt",
            "name": "projects/i-backbone-307006/locations/global/agent/intents/9416bbd6-b6b2-472e-b1c1-119a3e368726"
        },
        "parameters": {
            "topic": "login",
            "password": "`12",
            "username": "elife"
        },
        "sentimentAnalysisResult": {
            "queryTextSentiment": {
                "magnitude": 0.1,
                "score": 0.1
            }
        },
        "intentDetectionConfidence": 0.3,
        "queryText": "`12",
        "languageCode": "en",
        "allRequiredParamsPresent": true,
        "outputContexts": [
            {
                "parameters": {
                    "no-input": 0.0,
                    "topic": "login",
                    "username": "elife",
                    "password.original": "`12",
                    "username.original": "elife",
                    "loggedin_user": "elife",
            

127.0.0.1 - - [27/Mar/2021 05:30:37] "POST /webhook/ HTTP/1.1" 200 -


Request:
{
    "queryResult": {
        "intent": {
            "displayName": "Topic",
            "name": "projects/i-backbone-307006/locations/global/agent/intents/b360717e-7a9f-481c-9238-9bcd45834399"
        },
        "parameters": {
            "topic": [
                "movie recommendation"
            ]
        },
        "sentimentAnalysisResult": {
            "queryTextSentiment": {
                "magnitude": 0.6,
                "score": 0.6
            }
        },
        "intentDetectionConfidence": 0.74934524,
        "queryText": "movie recommendation",
        "languageCode": "en",
        "fulfillmentMessages": [
            {
                "text": {
                    "text": [
                        ""
                    ]
                }
            }
        ],
        "outputContexts": [
            {
                "parameters": {
                    "no-input": 0.0,
                    "no-match": 0.0,
                    "topic.original": [
     